In [31]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
df = pd.read_csv('./Drama_List.csv')
df[:5]

,Unnamed: 0,Title_english,Rating,Genre,Title_korean,Synopsis,Actor,Director,Writer,Tags
0,0,Cat's Bar,7.8,"Comedy, Romance, Drama, Supernatural",Taste of Cat,"After their ""butler""gets hurt, 3 mysterious ca...",Choi Hui Seung,Taste of Cat,Comedy,"Web Series, Short Length Series, Playful Male ..."
1,1,I Have a Lover,7.8,"Romance, Drama, Melodrama",애인 있어요,A coincidental switch of cars leads to a switc...,"Kim Hyun Joo, Ji Jin Hee, Park Han Byul, Lee K...",Bae Yoo Mi,Choi Moon Seok,"Amnesia, Birth Secret, Strong Female Lead, Sec..."
2,2,Dear My Name,7.8,"Romance, Fantasy",나의 이름에게,"In an alternate world, the name of one's soulm...",NaN,Kim Soo Jung,Romance,"Soulmates, Misunderstanding, Tomboy, Miniserie..."
3,3,The Flower in Prison,7.8,"Historical, Law, Drama, Melodrama",옥중화,"A story about Ok Nyeo, a talented girl born an...","Jin Se Yeon, Go Soo, Jung Joon Ho, Park Joo Mi...",Choi Wan Kyu,Choi Jung Gyu,"Strong Female Lead, Bad/Wrong Tag, Historical ..."
4,4,The Royal Gambler,7.8,"Action, Historical, Comedy, Drama",대박,Baek Dae Gil and King Yeong Jo enter a match. ...,"Jang Geun Suk, Yeo Jin Goo, Jeon Kwang Ryul, C...",Nam Gun,Kwon Soon Gyu,"Historical Fiction, Royalty, Gambling, Family ..."


In [33]:
# 줄거리 텍스트에 대해 전처리를 수행하고 'cleaned'라는 새로운 열에 저장

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['Synopsis'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(make_lower_case)
df['cleaned'] = df.cleaned.apply(remove_stop_words)
df['cleaned'] = df.cleaned.apply(remove_punctuation)
df['cleaned'] = df.cleaned.apply(remove_html)

In [34]:
# Nan값으로 바꾸어
df['cleaned'].replace('', np.nan, inplace=True)
df = df[df['cleaned'].notna()]
print('전체 문서의 수 :',len(df))

전체 문서의 수 : 1352


In [35]:
#문서별 번호 다시 정렬
df = df.reset_index()
df

,index,Unnamed: 0,Title_english,Rating,Genre,Title_korean,Synopsis,Actor,Director,Writer,Tags,cleaned
0,0,0,Cat's Bar,7.8,"Comedy, Romance, Drama, Supernatural",Taste of Cat,"After their ""butler""gets hurt, 3 mysterious ca...",Choi Hui Seung,Taste of Cat,Comedy,"Web Series, Short Length Series, Playful Male ...",butler gets hurt mysterious cats start running...
1,1,1,I Have a Lover,7.8,"Romance, Drama, Melodrama",애인 있어요,A coincidental switch of cars leads to a switc...,"Kim Hyun Joo, Ji Jin Hee, Park Han Byul, Lee K...",Bae Yoo Mi,Choi Moon Seok,"Amnesia, Birth Secret, Strong Female Lead, Sec...",coincidental switch cars leads switch fates tw...
2,2,2,Dear My Name,7.8,"Romance, Fantasy",나의 이름에게,"In an alternate world, the name of one's soulm...",NaN,Kim Soo Jung,Romance,"Soulmates, Misunderstanding, Tomboy, Miniserie...",alternate world name one s soulmate magically ...
3,3,3,The Flower in Prison,7.8,"Historical, Law, Drama, Melodrama",옥중화,"A story about Ok Nyeo, a talented girl born an...","Jin Se Yeon, Go Soo, Jung Joon Ho, Park Joo Mi...",Choi Wan Kyu,Choi Jung Gyu,"Strong Female Lead, Bad/Wrong Tag, Historical ...",story ok nyeo talented girl born raised prison...
4,4,4,The Royal Gambler,7.8,"Action, Historical, Comedy, Drama",대박,Baek Dae Gil and King Yeong Jo enter a match. ...,"Jang Geun Suk, Yeo Jin Goo, Jeon Kwang Ryul, C...",Nam Gun,Kwon Soon Gyu,"Historical Fiction, Royalty, Gambling, Family ...",baek dae gil king yeong jo enter match baek da...
...,...,...,...,...,...,...,...,...,...,...,...,...
1347,1616,1616,Time,7.8,"Mystery, Romance, Melodrama",시간,This is the story of two polar opposite people...,"Kim Jung Hyun, Seo Hyun, Kim Joon Han, Hwang S...",Choi Ho Chul,Jang Joon Ho,"Slight Romance, Revenge, Rich Man/Poor Woman, ...",story two polar opposite people come together ...
1348,1617,1617,Longing Heart,7.8,"Mystery, Comedy, Romance, Fantasy",애간장,A man named Shin Woo regrets letting his first...,"Lee Jung Shin, Seo Ji Hoon, Lee Yeol Eum, Kim ...",Min Yeon Hong,Lee Jung Hoon,"Doctor Female Lead, Secret, Hidden Identity, M...",man named shin woo regrets letting first love ...
1349,1618,1618,Love with Flaws,7.8,"Comedy, Romance, Drama, Family",하자있는 인간들,This romantic comedy follows a woman who has a...,"Oh Yeon Seo, Ahn Jae Hyun, Kim Seul Gi, Gu Won...",Oh Jin Suk,Ahn Shin Yoo,"Misunderstanding, Eccentric Male Lead, Trauma,...",romantic comedy follows woman aversion flower ...
1350,1619,1619,The Ghost Detective,7.8,"Thriller, Mystery, Horror, Supernatural",오늘의 탐정,"Lee Da Il is a private investigator, who catch...","Choi Daniel, Park Eun Bin, Lee Ji Ah, Kim Won ...",Lee Jae Hoon,Kang Soo Yeon,"Strong Female Lead, Female Antagonist, Ghost-s...",lee da il private investigator catches ghosts ...


In [36]:
# 수행 결과 확인
df['cleaned'][:5]

0    butler gets hurt mysterious cats start running...
1    coincidental switch cars leads switch fates tw...
2    alternate world name one s soulmate magically ...
3    story ok nyeo talented girl born raised prison...
4    baek dae gil king yeong jo enter match baek da...
Name: cleaned, dtype: object

In [37]:
# 토큰화를 수행하여 corpus라는 리스트에 저장함. corpus로 Word2Vec을 훈련할 예정

corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

### 2. 사전 훈련된 워드 임베딩 가져와 사용하기
- 데이터가 충분하지 않다면 사전 훈련된 모델을 사용하고 데이터가 충분하다면 데이터를 학습시켜 모델을 만들면 됨

In [38]:
# 이 경우는 데이터가 충분치 않아서 구글에서 제공하는 모델을 사용하고자 함
import gensim.downloader as api

word2vec_model = api.load('word2vec-google-news-300')

### 3. 단어 벡터의 평균 구하기
- 각 문서에 존재하는 단어들의 벡터값의 평균을 구하여 해당 문서의 벡터값을 계산

In [39]:
def get_document_vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.index_to_key:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더함
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠줌
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [40]:
list(df['cleaned'])

['butler gets hurt mysterious cats start running bar make living nabi byeol janggun three cats turn humans help customers offering special drinks make revisit memories',
 'coincidental switch cars leads switch fates twins separated yers ago hae kang elder sister successful lawyer married jin eon successor top pharmaceutical company love marriage shattered death daughter husband s affair seol ri ultimately leads divorce divorce officially declared yong ki switches cars hae kang avoid followed people life yong ki younger sister pregnant widow trying uncover secrets behind husband s sudden death fateful encounter hae kang ends horrible accident in switched car makes lose memories beak seok saves allows live family till recovers mistakes first love yong ki hae kang yong ki longer cold reserved person instead warm cheerful caring seol ri jin eon talks marriage accidentally runs yong ki shocked first gradually begins suspecting wife falls newly discovered bright self shaken arrival too sourc

In [41]:
def get_document_vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [42]:
document_embedding_list = get_document_vectors(df['cleaned'])
print('문서 벡터의 수 :',len(document_embedding_list))

문서 벡터의 수 : 1352


<ipython-input-41-69ed8f6fd736>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in word2vec_model.wv.vocab:


# 4. 추천 시스템 구현하기
- 입력한 드라마의 줄거리와 가장 유사한 드라마을 추천하는 것을 목표로 함

In [43]:
# 각 문서 벡터 간 코사인 유사도를 계산
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (1352, 1352)


In [66]:
def recommendations(title):
    drama= df[['Title_korean']]
    

    # 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(df.index, index = df['Title_korean']).drop_duplicates()    
    idx = indices[title]

    # 입력된 드라마와 줄거리(document embedding)가 유사한 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 드라마 5편의 인덱스
    drama_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = pd.DataFrame(drama.iloc[drama_indices].reset_index(drop=True))
    print('유사한 줄거리로 다음과 같은 드라마가 있습니다:')
    return recommend

    

In [67]:
df['Title_korean']

0        Taste of Cat
1              애인 있어요
2             나의 이름에게
3                 옥중화
4                  대박
            ...      
1347               시간
1348              애간장
1349         하자있는 인간들
1350           오늘의 탐정
1351    My Sassy Girl
Name: Title_korean, Length: 1352, dtype: object

In [68]:
recommendations("나의 아저씨")

유사한 줄거리로 다음과 같은 드라마가 있습니다:


,Title_korean
0,Phoenix
1,Save Me 2
2,아름다운 나의 신부
3,내 사위의 여자
4,한 번 다녀왔습니다


In [86]:
recommendations("나의 아저씨").values[0][0]

유사한 줄거리로 다음과 같은 드라마가 있습니다:


'Phoenix'

In [47]:
#recommendations = input('참고할 드라마를 입력하세요: ')

In [72]:
from webdriver_manager.chrome import ChromeDriverManager

In [76]:
from selenium import webdriver

In [81]:
from bs4 import BeautifulSoup

In [82]:
urls = 'https://mydramalist.com/' #랭킹 전체 페이지
res = requests.get(urls)
soup = BeautifulSoup(res.content, 'html.parser')
    
driver.get(urls)
driver.maximize_window()
driver.implicitly_wait(3)
driver.find_element_by_class_name('form-control p-x b-a rounded').send_keys('Phoenix')
driver.find_element_by_class_name('form-control p-x b-a rounded').send_keys(Keys.ENTER)
css_selector = "#mdl-1167 > div > div > div.col-xs-9.row-cell.content"
element = driver.find_element_by_css_selector(css_selector)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".form-control p-x b-a rounded"}
  (Session info: chrome=102.0.5005.61)


In [78]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://mydramalist.com/'
driver.get(url)

driver.find_element_by_class_name('form-control p-x b-a rounded').send_keys('Phoenix')
driver.find_element_by_class_name('form-control p-x b-a rounded').send_keys(Keys.ENTER)
css_selector = "#mdl-1167 > div > div > div.col-xs-9.row-cell.content"
element = driver.find_element_by_css_selector(css_selector)
# driver.find_element_by_css_selector('#hdtb-msb > div:nth-child(1) > div > div:nth-child(2) > a').click()
# time.sleep(2)

# images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
# count = 1

# for image in images:
#     image.click() # 클래스 찾아 클릭
#     time.sleep(3)
#     imgUrl = driver.find_element_by_css_selector(".n3VNCb").get_attribute("src")
#   #이미지주소 src 다운로드
#     urllib.request.urlretrieve(imgUrl, 'images/jt{0}.jpg'.format(count))
#     count = count + 1
#     if count==31:
#         break



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/hayeon/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".form-control p-x b-a rounded"}
  (Session info: chrome=102.0.5005.61)
